

## 0. 라이브러리 설치



In [ ]:
!pip install pycaret==2.3.10

     |████████████████████████████████| 320 kB 4.2 MB/s 
     |████████████████████████████████| 1.3 MB 44.0 MB/s 
     |████████████████████████████████| 2.0 MB 34.0 MB/s 
     |████████████████████████████████| 16.8 MB 14.2 MB/s 
     |████████████████████████████████| 262 kB 50.7 MB/s 
     |████████████████████████████████| 1.7 MB 42.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 5.2 MB/s 
     |████████████████████████████████| 118 kB 42.5 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 6.8 MB 43.4 MB/s 
     |████████████████████████████████| 167 kB 53.6 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 636 kB 48.8 MB/s 
     |████████████████████████████████| 690 kB 51.7 MB/s 
     |███████████████████

In [ ]:
!pip install jinja2==3.1.2

     |████████████████████████████████| 133 kB 4.2 MB/s 
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install xgboost==1.6.0

     |████████████████████████████████| 193.7 MB 28 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


## 1. 데이터 불러오기
전처리된 데이터 csv파일로 저장해 구글 드라이브에 올려놓고 가져와 사용하는 방식
- index, occyp_type, FLAG_MOBIL 삭제
- gender, car, reality -> 0과 1로 변경
- DAYS_BIRTH -> age 컬럼 생성 후 삭제
- DAYS_EMPLOYED -> (-1)곱하고, 음수는 0으로 변경
- begin_month -> (-1) 곱해줌
- income_total -> 로그스케일 적용

In [ ]:
from google.colab import drive

drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
import io
import pandas as pd

In [ ]:
file = '/gdrive/MyDrive/Colab Notebooks/Semi_project_03/df_occyp예측.csv'
df = pd.read_csv(file)
df.drop(['index','DAYS_BIRTH','FLAG_MOBIL'], axis = 1, inplace = True)
df.head(3)

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_EMPLOYED,work_phone,phone,email,occyp_type,family_size,begin_month,credit,age
0,0,0,0,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,4709,0,0,0,Laborers,2.0,6.0,1.0,38
1,0,0,1,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,1540,0,0,1,Laborers,3.0,5.0,1.0,31
2,1,1,1,0,450000.0,Working,Higher education,Married,House / apartment,4434,0,1,0,Managers,2.0,22.0,2.0,52


In [ ]:
df

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,age
0,0,0,0,0,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,4709,1,0,0,0,Laborers,2.0,6.0,1.0,38
1,1,0,0,1,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,1540,1,0,0,1,Laborers,3.0,5.0,1.0,31
2,2,1,1,1,0,450000.0,Working,Higher education,Married,House / apartment,4434,1,0,1,0,Managers,2.0,22.0,2.0,52
3,3,0,0,1,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,2092,1,0,1,0,Sales staff,2.0,37.0,0.0,41
4,4,0,1,1,0,157500.0,State servant,Higher education,Married,House / apartment,2105,1,0,0,0,Managers,2.0,26.0,2.0,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,0,0,0,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,1984,1,0,0,0,Core staff,4.0,2.0,1.0,33
26453,26453,0,0,1,1,180000.0,Working,Higher education,Separated,House / apartment,2475,1,0,0,0,Managers,2.0,47.0,2.0,41
26454,26454,0,1,0,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,2015,1,0,0,0,Core staff,2.0,25.0,2.0,27
26455,26455,1,0,1,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,107,1,0,0,0,Laborers,1.0,59.0,2.0,27


## 2. pycaret 적용

In [ ]:
from pycaret.classification import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
model = setup(data= df,
              target='credit',
              numeric_features=['DAYS_EMPLOYED','child_num'], # categoric으로 분류돼서 직접 지정
              train_size=0.7,
              session_id=42,
              fix_imbalance=True)

,Description,Value
0,session_id,42
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(26457, 18)"
5,Missing Values,False
6,Numeric Features,6
7,Categorical Features,11
8,Ordinal Features,False
9,High Cardinality Features,False


## 2-1. 분류 모델 비교

In [ ]:
# 평가 지표 추가
from sklearn.metrics import log_loss

add_metric('logloss','LogLoss',log_loss,greater_is_better=False,target='pred_proba')

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7f2fbcee8050>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [ ]:
# svm과 ridge 모델은 predict_proba를 지원하지 않아 제외
top_3_models = compare_models(sort='logloss', n_select=3, exclude=['svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7036,0.7011,0.4727,0.6791,0.6556,0.2974,0.3403,0.7739,26.234
lightgbm,Light Gradient Boosting Machine,0.7045,0.6932,0.4525,0.6861,0.6442,0.2759,0.3365,0.7802,2.121
gbc,Gradient Boosting Classifier,0.6934,0.6422,0.4162,0.6626,0.6143,0.2214,0.2981,0.8407,22.573
lda,Linear Discriminant Analysis,0.4481,0.6065,0.3917,0.5505,0.4789,0.0989,0.1053,1.0719,0.536
lr,Logistic Regression,0.4043,0.5448,0.3640,0.4820,0.4166,0.0536,0.0566,1.0804,1.752
ada,Ada Boost Classifier,0.6925,0.6043,0.4111,0.6101,0.6097,0.2134,0.2958,1.0901,1.999
dummy,Dummy Classifier,0.1203,0.5000,0.3333,0.0145,0.0258,0.0000,0.0000,1.0986,0.213
nb,Naive Bayes,0.4230,0.5998,0.3849,0.5446,0.4496,0.0848,0.0937,1.1006,0.241
rf,Random Forest Classifier,0.6931,0.7408,0.5450,0.6755,0.6811,0.3671,0.3706,1.1017,5.299
et,Extra Trees Classifier,0.6666,0.7041,0.5227,0.6494,0.6554,0.3159,0.3185,2.6156,5.847


In [ ]:
# xgboost모델 예측 결과값
predict_model(top_3_models[0])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Extreme Gradient Boosting,0.6973,0.7129,0.4662,0.6712,0.6456,0.2899,0.3355,0.769


,child_num,income_total,DAYS_EMPLOYED,family_size,begin_month,age,gender_0,car_0,reality_0,income_type_Commercial associate,...,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,credit,Label,Score
0,0.0,315000.0,545.0,1.0,11.0,28.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.6167
1,1.0,216000.0,1782.0,2.0,7.0,28.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.5268
2,0.0,238500.0,7095.0,2.0,53.0,41.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.6310
3,0.0,112500.0,0.0,2.0,14.0,66.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.6974
4,1.0,247500.0,1931.0,3.0,16.0,35.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.6925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7933,0.0,135000.0,0.0,1.0,-0.0,63.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0,0.6589
7934,0.0,270000.0,1674.0,2.0,29.0,26.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.4932
7935,0.0,135000.0,1259.0,2.0,24.0,28.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.4724
7936,0.0,117000.0,0.0,2.0,39.0,63.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.6034


## 2-2. 분류 모델 튜닝

In [ ]:
tuned_top3 = [tune_model(i, optimize='logloss') for i in top_3_models]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.6884,0.6949,0.4931,0.6599,0.6575,0.3042,0.3216,0.7990
1,0.7030,0.7141,0.5050,0.6748,0.6723,0.3364,0.3562,0.7826
2,0.6917,0.7160,0.4970,0.6613,0.6613,0.3152,0.3323,0.7930
3,0.6798,0.6980,0.4768,0.6443,0.6467,0.2854,0.3025,0.8030
4,0.6852,0.7157,0.4827,0.6519,0.6530,0.2964,0.3143,0.7916
5,0.6982,0.7284,0.5023,0.6684,0.6683,0.3296,0.3471,0.7653
6,0.7127,0.7308,0.5108,0.6886,0.6803,0.3507,0.3762,0.7610
7,0.6874,0.7086,0.4790,0.6525,0.6520,0.2944,0.3149,0.7966
8,0.6922,0.7099,0.4854,0.6588,0.6571,0.3068,0.3271,0.7838


In [ ]:
top_3_models

[XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
               missing=nan, monotone_constraints='()', n_estimators=100,
               n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
               predictor='auto', random_state=42, reg_alpha=0, ...),
 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=100, n_job

In [ ]:
tuned_top3

[XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=2,
               missing=nan, monotone_constraints='()', n_estimators=170,
               n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
               predictor='auto', random_state=42, reg_alpha=10, ...),
 LGBMClassifier(bagging_fraction=0.8, bagging_freq=3, boosting_type='gbdt',
                class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
                importance_type='split', learning_rate=0.2, max_depth=-1,
                min_child_samples=6, min_child_wei

## 2-3. 분류 모델 블렌딩

In [ ]:
blend_top_3 = blend_models(estimator_list=tuned_top3, fold=5, optimize='logloss')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.7087,0.7079,0.4912,0.6847,0.6685,0.3248,0.3582,0.7628
1,0.6974,0.7077,0.4739,0.6661,0.6542,0.2957,0.3281,0.7711
2,0.7041,0.7212,0.4871,0.6788,0.6637,0.3144,0.3483,0.7594
3,0.7073,0.7167,0.4809,0.6795,0.6630,0.3158,0.3530,0.7620
4,0.7121,0.7182,0.4959,0.6898,0.6720,0.3319,0.3669,0.7538
Mean,0.7059,0.7143,0.4858,0.6798,0.6643,0.3165,0.3509,0.7618
Std,0.0050,0.0055,0.0077,0.0079,0.0060,0.0122,0.0130,0.0056


In [ ]:
# 블렌딩 모델로 예측
pred = predict_model(blend_top_3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.706,0.7327,0.4953,0.6837,0.6656,0.3297,0.3645,0.7492


## 2.4 마무리

In [ ]:
# finalize_model : 전체 데이터로 마지막 학습을 진행함
final_blended = finalize_model(blend_top_3)

In [ ]:
final_blended

VotingClassifier(estimators=[('xgboost',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            callbacks=None, colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0.7,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=0,
                                            gpu_id=-1, grow_policy='depthwise',
                                            importance_type=None,
                                            interaction_constraints='',
                                            learning_rate=0.3, max_bin=256,...
                                                         min_impurity_decrease=0.001,
                                                         min_impurity_spli

In [ ]:
# 최종 모델로 예측
predict_model(final_blended)

## 2.5 모델 저장

In [ ]:
save_model(final_blended, '/gdrive/MyDrive/Colab Notebooks/Semi_project_03/final_blended_model_occupy_SMOTE')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=['index'],
                                       ml_usecase='classification',
                                       numerical_features=['DAYS_EMPLOYED',
                                                           'child_num'],
                                       target='credit', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_...
                                                                           min_impurity_decrease=0.001,
                                                                           min_impurity_split=None,
                                            